In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow_text
import tensorflow_text as text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 34.5 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras import Sequential
from keras.models import load_model
from keras.optimizers import Adam, SGD
from keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, Input, GlobalAveragePooling1D, Flatten
import tensorflow_hub as hub
from keras import Model
from sklearn.metrics import classification_report, confusion_matrix
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings("ignore")




def built_model_Bert():
    text_input = Input(shape=(), dtype = tf.string, name = 'text')
    preprocessing_layer = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3", name ='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2", trainable=False, name = "Bert-encoder")
    outputs = encoder(encoder_inputs)
    net = outputs["pooled_output"]
    net = Dense(400, activation ='relu')(net)
    net = Dropout(0.1)(net)
    net = Dense(200, activation ='relu')(net)
    net = Dropout(0.1)(net)
    net = Dense(100, activation ='relu')(net)
    net = Dropout(0.1)(net)
    net = Dense(50, activation ='relu')(net)
    net = Dropout(0.1)(net)
    net = Dense(12, activation ='sigmoid', name = 'classifier')(net)
    return Model(inputs = text_input, outputs = net)

Bert_model = built_model_Bert()




In [31]:
Bert_model.load_weights('/content/drive/MyDrive/MutilEmotion_Bert/model_small_Bert.h5')
Bert_model.summary()




Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_word_ids': (None,                                           
                              128),                                                               
                              'input_type_ids': (None,                                            
                             128)}                                                                
                                                                                            

In [36]:
label_class = ["anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust", "neutral"]

text_input = '''
Tai was extremely angry because his girlfriend fell in love with another man,
              that's why he was very sad the next day and at the same time he lost faith in life
              '''

def predict_sentiment(input_text):
    sentiment = []
    pred_result = Bert_model.predict([input_text])
    pred_results = list(tf.round(pred_result).numpy()[0])
    for v, l in zip(pred_results, label_class):
      if v == 1.0:
        sentiment.append(l)

    return ', '.join(sentiment)

predict_sentiment(text_input)



1/1 [==============================] - 0s 69ms/step


'anger, disgust'

In [ ]:
!pip install gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 7.2 MB/


1.	I'm so excited about the upcoming event
2.	His kindness brought tears to my eyes.
3.	Winning the competition has filled me with joy
4.	I really hope you give my present




In [37]:
demo = gr.Interface(
    fn=predict_sentiment,
    inputs= "text",
    outputs= "text",
    live=True
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a65d51fbde7b5984a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
